# MD5 Collider Demo

This notebook demonstrates how to use `collider.py` to:
- compute an MD5 digest via SAT (end-to-end)
- solve a single MD5 chunk
- set up an unconstrained search for an collision after a single round of MD5
- set up a (very) constrained search for an MD5 collision

We will rely on `MD5Collider` and the reference `MD5` implementation in `md5.py`.


In [2]:
from collider import MD5Collider
from md5 import MD5

def to_hex128(n: int) -> str:
    return f"0x{n:032x}"


In [8]:
# Solve a single MD5 chunk (64 bytes) via SAT
msg = b"Hello, World!"
padded = MD5.md5_padded(msg)
print(f"Original message: {msg!r}")
print(f"Padded length (bytes): {len(padded)}")

# Reference digest on the same padded bytes
ref = MD5()
true_digest = ref.md5_digest(padded)
print("Reference digest:", to_hex128(true_digest))

# Build SAT model and constrain one chunk, then solve
collider = MD5Collider(padded)
collider.solve_md5_chunk(0)

sat = collider.solver.solve()
if not sat:
    print("UNSAT: No satisfying assignment for this chunk.")
else:
    model = collider.solver.get_model()
    x, digest = collider.process_solution(model)
    print("SAT: Model found.")
    print("Recovered input equals padded:", x == padded)
    print("SAT-derived digest:", to_hex128(digest))
    print("Digest matches reference:", digest == true_digest)


Original message: b'Hello, World!'
Padded length (bytes): 64
Reference digest: 0x65a8e27d8879283831b664bd8b7f0ad4
SAT: Model found.
Recovered input equals padded: True
SAT-derived digest: 0x65a8e27d8879283831b664bd8b7f0ad4
Digest matches reference: True


In [6]:
base = b"Hello, World!"
padded_base = MD5.md5_padded(base)
target_digest = MD5().md5_digest(padded_base, num_rounds=1)
print("Target digest:", to_hex128(target_digest))

collider = MD5Collider(None, target_digest=target_digest)

# Interrupt the solver after a short time budget
sat, result = collider.solve_md5(num_rounds=1)

if sat:
    x, digest = result
    print("Solver returned SAT within time limit.")
    print("Digest matches target:", digest == target_digest)
    print("Input differs from original:", x != padded_base)
    print("Orig:\t", padded_base)
    print("Found:\t", x)
    print("Found digest:", to_hex128(digest))


Target digest: 0xf9134a103ac1d8f94193cea4ea338853
Solver returned SAT within time limit.
Digest matches target: True
Input differs from original: True
Orig:	 b'Hello, World!\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00h\x00\x00\x00\x00\x00\x00\x00'
Found:	 b'\x8e\x88\xa0\x00\xf0\xcb<\xac\xb80\xd1\t\xd0#!\x01\xd3\x8f\x11+\xff/w\xe9l\x17*\xa6($\xeb\xddl\xd9j\xb4\xe1\tk\xa7Y1m\x9e\xef\x039OK+\x90\x12\x80\x0f\x82\x1d\x06\\\xe8I7\xe5\x82b'
Found digest: 0xf9134a103ac1d8f94193cea4ea338853


In [7]:
from threading import Timer

base = b"TEXTCOLLBYfGiJUETHQ4hEcKSMd5zYpgqf1YRDhkmxHkhPWptrkoyz28wnI9V0aHeAuaKnak"
padded_base = MD5.md5_padded(base)
target_digest = MD5().md5_digest(padded_base)
print("Target digest:", to_hex128(target_digest))

# Allow a few input bits to differ; this mirrors the test harness idea

def interrupt(s):
    s.interrupt()

for i in range(512):
    exclude_bits = [i]
    collider = MD5Collider(padded_base, target_digest=target_digest, exclude_input_bits=exclude_bits)

    # Interrupt the solver after a short time budget
    sat, result = collider.solve_md5()

    if sat:
        x, digest = result
        print("Solver returned SAT within time limit.")
        print("Digest matches target:", digest == target_digest)
        print("Input differs from original:", x != padded_base)
        print("Orig:\t", padded_base)
        print("Found:\t", x)
        print("Found digest:", to_hex128(digest))
        break


Target digest: 0xfaad49866e9498fc1719f5289e7a0269
Solver returned SAT within time limit.
Digest matches target: True
Input differs from original: True
Orig:	 b'TEXTCOLLBYfGiJUETHQ4hEcKSMd5zYpgqf1YRDhkmxHkhPWptrkoyz28wnI9V0aHeAuaKnak\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x02\x00\x00\x00\x00\x00\x00'
Found:	 b'TEXTCOLLBYfGiJUETHQ4hAcKSMd5zYpgqf1YRDhkmxHkhPWptrkoyz28wnI9V0aHeAuaKnak\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x02\x00\x00\x00\x00\x00\x00'
Found digest: 0xfaad49866e9498fc1719f5289e7a0269
